In [1]:
import pandas as pd
import matplotlib.pyplot as pyplot
import numpy as np

from sklearn.model_selection import StratifiedKFold

In [16]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
survived = train["Survived"]
train.info()

data = pd.concat([train.drop(["Survived"], axis=1) , test]) #train has an additional column called Survived, so there is a column mismatch

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [17]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB


In [19]:
data["Age"] = data.Age.fillna(data.Age.median())
data["Fare"] = data.Fare.fillna(data.Fare.median())
data = pd.get_dummies(data, columns=["Sex"], drop_first=True)
data = data[['Sex_male', 'Fare', 'Age','Pclass', 'SibSp']]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 5 columns):
Sex_male    1309 non-null uint8
Fare        1309 non-null float64
Age         1309 non-null float64
Pclass      1309 non-null int64
SibSp       1309 non-null int64
dtypes: float64(2), int64(2), uint8(1)
memory usage: 52.4 KB


In [20]:
X = data.iloc[:891]
test = data.iloc[891:]
y = survived # Series

X.head()

,Sex_male,Fare,Age,Pclass,SibSp
0,1,7.2500,22.0,3,1
1,0,71.2833,38.0,1,1
2,0,7.9250,26.0,3,0
3,0,53.1000,35.0,1,1
4,1,8.0500,35.0,3,0


In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

n_folds = 9
kf = StratifiedKFold(n_splits=n_folds, shuffle=True)
scores = np.zeros((n_folds, 4), dtype=np.float)

# Indexing by list of indices does not work for DataFrame [] syntax
X = X.values

In [29]:
for i, (train_index, test_index) in enumerate(kf.split(X,y)):
    y_test = y[test_index]
    m = RandomForestClassifier()
    m.fit(X[train_index], y[train_index]) 
    y_pred = m.predict_proba(X[test_index])
    print(y_pred.shape)
    print(type(y_pred[0][0]))
    print(type(y_test))
    
    p, r, f, _ = precision_recall_fscore_support(y_test, y_pred, average = 'macro')
#     print(type(result))
#     scores[i,:] = metric(y_test)

(99, 2)
<class 'numpy.float64'>
<class 'pandas.core.series.Series'>


ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets